In [1]:
import numpy as np
import time
from random import randint
from qiskit import QuantumCircuit,Aer

# np.random.seed(0)
n = 100

st = time.process_time()
#Alice's bits
alice_bits = np.random.randint(2,size=n)
alice_bases = np.random.randint(2,size=n)

# print(alice_bits)

def encodeMessage(instring, bases):
    """If bases=0 -> encode in X(Hadamard gate) basis
        If bases=1 -> encode in Z(Pauli X gate) basis
        Returns an array consisting of circuit object"""
    message =[]
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0:
            if instring[i]==0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        else:
            if instring[i] == 0:
                pass
            else:
                qc.x(0)
        qc.barrier()        
        message.append(qc)
    return message

codedm = encodeMessage(alice_bits,alice_bases)
# codedm[1].draw()

bob_bases = np.random.randint(2,size=n)

def decodeMessage(bases,message):    
    measured = []
    for i in range(n):
        if bases[i] == 0: #Measure in X-basis
            message[i].h(0)
            message[i].measure(0,0)
        if bases[i] == 1: #Measure in Z-basis
            message[i].measure(0,0) 
    
        simulator = Aer.get_backend('aer_simulator')
        results = simulator.run(message[i],shots=1,memory=True).result()
        bit = int(results.get_memory()[0])
        measured.append(bit)
        
    return measured

bob_bits = decodeMessage(bob_bases,codedm)
# print(bob_bits)



def getKey(a_bases, b_bases, bits):
    thekey = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add this to the list of 'good' bits
            thekey.append(bits[q])
    return thekey

alice_key = getKey(alice_bases,bob_bases,alice_bits)
bob_key = getKey(alice_bases,bob_bases,bob_bits)

def sampling(bits, selector):
    sample = []
    for i in selector:
        i = np.mod(i, len(bits)) #<-- Dont' really understand why this
        sample.append(bits.pop(i))
    return sample

selector = np.random.randint(n,size = 15)
bob_sample = sampling(bob_key,selector)
# print(bob_sample)
alice_sample = sampling(alice_key,selector)
# print(alice_sample)
et = time.process_time()
print("time taken in ms: ",str((et-st)*1000))

print(alice_key==bob_key)
print("Alice's key: ",alice_key)
print("Bob's key: ",bob_key)

time taken in ms:  234.375
True
